<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/BDLE_DM1_2022_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BDLE 2022

date du document  :  07/10/2022

# TP2 et 3 Fenêtres


SQL avec fonctions sur des fenêtres

## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [ ]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/BDLE_S3_bis_COLAB_Notebooks/bdle/DM1"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 49.0 MB/s 


Démarrer la session spark

In [ ]:
import os

# !find /usr/local/lib -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1665329206786


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le résultat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = r.lower().startswith('select')
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [ ]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [ ]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

In [ ]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi 
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15/userVisits-Toro.csv save in : /local/data/userVisits-Toro.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15/POI-Toro.csv save in : /local/data/POI-Toro.csv


['userVisits-Toro.csv', 'POI-Toro.csv']

### Les visites

On considère un fichier décrivant les check-ins d'utilisateurs et leur déplacements. Une ligne correspond à une photo prise par un utilisateur. On connait le lieu et la date d'une photo. 
Les photos consécutives durant une journée forment une séquence dont on connait l'identifiant (seqID). 

Lire les 2 premières lignes du  fichier csv en python. 
Est ce que le fichier a une ligne d'entête ?
Quel caractère délimite deux valeurs consécutives dans une ligne de données ?

In [ ]:
f = open(local_dir + "/" + "userVisits-Toro.csv", "r")
print(f.readline()); print(f.readline())

"photoID";"userID";"dateTaken";"poiID";"poiTheme";"poiFreq";"seqID"

7941504100;"10007579@N00";1346844688;30;"Structure";1538;1



**Réponse :** Le fichier a une entête. Deux valeurs consécutives sont délimité par le caractère ";".

Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [ ]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [ ]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(4)
user_visits.printSchema()
display(user_visits)

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 4 rows

root
 |-- photoID: long (nullable = true)
 |-- userID: string (nullable = true)
 |-- date: long (nullable = true)
 |-- poiID: integer (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: integer (nullable = true)
 |-- seqID: integer (nullable = true)



,photoID,userID,date,poiID,poiTheme,poiFreq,seqID
0,7941504100,10007579@N00,1346844688,30,Structure,1538,1
1,4886005532,10012675@N05,1142731848,6,Cultural,986,2
2,4886006468,10012675@N05,1142732248,6,Cultural,986,2
3,4885404441,10012675@N05,1142732373,6,Cultural,986,2
4,4886008334,10012675@N05,1142732445,6,Cultural,986,2
...,...,...,...,...,...,...,...
95,2654929774,10014440@N06,1215593613,25,Shopping,1701,10
96,2654104231,10014440@N06,1215593634,25,Shopping,1701,10
97,2654930912,10014440@N06,1215593650,25,Shopping,1701,10
98,2654105249,10014440@N06,1215593655,25,Shopping,1701,10


Nombre de photos, nombre de séquences et nombre d'évenements *un user a visité un POI* déterminés par un couple (séquence, POI)

In [ ]:
%%sql
select count(*) as nbPhotos, count(distinct seqID) as nbSequences, count(distinct seqID, poiID) as nb_visites
from user_visits

,nbPhotos,nbSequences,nb_visites
0,39419,6057,7607


Les POI avec la plus grande frequence précalculée *poiFreq*

In [ ]:
%%sql
select distinct poiID, poiFreq 
from user_visits
order by poiFreq desc

,poiID,poiFreq
0,11,4142
1,22,3619
2,21,3594
3,16,3553
4,1,3506
5,4,3056
6,7,2064
7,23,1874
8,8,1736
9,25,1701


Les POI avec la plus grande fréquence (ici la "fréquence bis" est le nombre de photos prises à un POI)

In [ ]:
%%sql
select poiID, max(poiFreq) as poiFreq, count(*) as poiFreqBis
from user_visits
group by poiID
order by poiFreqBis desc

,poiID,poiFreq,poiFreqBis
0,11,4142,4139
1,22,3619,3603
2,21,3594,3591
3,16,3553,3553
4,1,3506,3506
5,4,3056,3056
6,7,2064,2053
7,23,1874,1866
8,8,1736,1736
9,25,1701,1701


### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [ ]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [ ]:
%%sql
cache table POI;

SELECT * 
FROM POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [ ]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

,theme
0,Amusement
1,Beach
2,Cultural
3,Shopping
4,Sport
5,Structure


## Exercice 1

#### 1) Identifier les thèmes

Définir la table Theme1(theme_id, name) avec les numéros de thème commençant à 1 pour les noms de thème triés dans l'ordre croissant.

In [ ]:
%%sql

create or replace temp view Theme1 as
  select theme
    , row_number() over (order by theme) as theme_id
  from Themes;

select * from Theme1

,theme,theme_id
0,Amusement,1
1,Beach,2
2,Cultural,3
3,Shopping,4
4,Sport,5
5,Structure,6


#### 2) Classement des séquences par leur plus grand nombre de POI **distincts**

Définir la table TopSeq(seqID, nbPOI, rang) avec rang=1 pour la séquence ayant le plus grand nbPOI.


In [ ]:
%%sql

create or replace temp view TopSeq as
  select *
    , rank() over (order by nbPOI desc) as rang
  from (
    select seqID
      , count(distinct poiID) as nbPOI
    from user_visits
    group by seqID
  );
  
select * from TopSeq

,seqID,nbPOI,rang
0,298,13,1
1,4961,10,2
2,4351,9,3
3,5964,9,3
4,5369,8,5
...,...,...,...
95,5183,4,60
96,5539,4,60
97,813,4,60
98,2340,4,60


#### 2) Identifier les check-ins




2a) Définir la table Visite1 telle qu'il n'y ait pas de doublons sur le triplet (seqID, poiID, date)

In [ ]:
%%sql
create or replace temp view Visite1 as
  select distinct seqID, poiID, date
  from user_visits;

select *
from Visite1
where seqID in (298, 510)
order by seqID, date, poiID;

,seqID,poiID,date
0,298,22,1371514467
1,298,7,1371514471
2,298,7,1371514472
3,298,23,1371516913
4,298,28,1371516914
5,298,28,1371516915
6,298,1,1371517315
7,298,29,1371517318
8,298,30,1371517318
9,298,30,1371517319


2b) Montrer qu'il existe des séquences où une même date est associée à plusieurs POI distincts. 
Indication : utiliser une requete de regroupement.

In [ ]:
%%sql
select seqID
  , date
  , count(distinct poiID) as nb_POI_meme_date
from Visite1
group by seqID, date
having nb_POI_meme_date > 1
order by nb_POI_meme_date desc, seqID

,seqID,date,nb_POI_meme_date
0,816,1291341600,3
1,822,1292896800,3
2,905,1317952800,3
3,1225,228664800,3
4,4271,1092646926,3
5,5497,1169172000,3
6,271,778341600,2
7,298,1371519708,2
8,298,1371519010,2
9,298,1371517318,2


2c) A partir de la table précédente, définir la table Visite2(seqID, poiID, date, num) avec *num* étant le numéro d'ordre dans une séquence.
Les POI visités à une même date sont triés dans l'ordre croissant de poiID.

In [ ]:
%%sql

create or replace temp view Visite2 as
  select seqID
    , date
    , poiID
    , rank() over (partition by seqID order by date, poiID) as num
  from Visite1;

select * from Visite2
where seqID in (298, 510)
order by num


,seqID,date,poiID,num
0,510,1184804839,28,1
1,298,1371514467,22,1
2,298,1371514471,7,2
3,510,1184804845,28,2
4,298,1371514472,7,3
5,510,1184805016,28,3
6,510,1184805075,28,4
7,298,1371516913,23,4
8,298,1371516914,28,5
9,510,1184805080,28,5


#### 3) Identifier les Visites de POI

Définir la table Visite3(userID, seqID, poiID, poiPosition) telle que la  poiPosition vaut *i* pour le ième POI visité dans une séquence.

Indications:

La visite d'un POI correspond à toutes les photos consécutives prises à ce POI.

Un même POI peut apparaitre **plusieurs** fois à des positions différentes dans une séquence si au moins un autre POI a été visité entre temps dans la séquence.


##### 3a) POI précédent
Commencer par associer chaque événement avec le POI précédent dans la séquence.
Indication penser aux fonctions lag() ou first()

In [ ]:
%%sql
create or replace temp view Visite3a as
 select *
   , lag(poiID, 1, null) over (partition by seqID order by date, poiID ) as precedent
  from Visite2;

select * from Visite3a
where seqID in (298, 510)
order by seqID, date;


,seqID,date,poiID,num,precedent
0,298,1371514467,22,1,NaN
1,298,1371514471,7,2,22.0
2,298,1371514472,7,3,7.0
3,298,1371516913,23,4,7.0
4,298,1371516914,28,5,23.0
5,298,1371516915,28,6,28.0
6,298,1371517315,1,7,28.0
7,298,1371517318,29,8,1.0
8,298,1371517318,30,9,29.0
9,298,1371517319,30,10,30.0


##### 3b) Début de visite
Ajouter un attribut *début* valant 1 pour le premier tuple d'une série de photos consécutives concernant le même POI et 0 sinon. 
Indication, penser à la syntaxe case when ...then ... else ... end

In [ ]:
%%sql
create or replace temp view Visite3b as
  select *
    , case
        when precedent is null or poiID = precedent then 0
        else 1
      end as debut
  from Visite3a;

select * from Visite3b
where seqID in (298, 510)
order by seqID, date;

,seqID,date,poiID,num,precedent,debut
0,298,1371514467,22,1,NaN,0
1,298,1371514471,7,2,22.0,1
2,298,1371514472,7,3,7.0,0
3,298,1371516913,23,4,7.0,1
4,298,1371516914,28,5,23.0,1
5,298,1371516915,28,6,28.0,0
6,298,1371517315,1,7,28.0,1
7,298,1371517318,29,8,1.0,1
8,298,1371517318,30,9,29.0,1
9,298,1371517319,30,10,30.0,0


##### 3c) Ordonner les POI visités
Définir la table Visite3 décrite au début de la question 3). Ajouter l'attribut poiPosition 

In [ ]:
%%sql
create or replace temp view Visite3 as
  select seqID
    , num
    , date
    , poiID
    , sum(debut) over (partition by seqID order by date, poiID) as poiPosition
  from Visite3b;

select * from Visite3
where seqID in (298, 510)
order by seqID, date, poiID;

,seqID,num,date,poiID,poiPosition
0,298,1,1371514467,22,0
1,298,2,1371514471,7,1
2,298,3,1371514472,7,1
3,298,4,1371516913,23,2
4,298,5,1371516914,28,3
5,298,6,1371516915,28,3
6,298,7,1371517315,1,4
7,298,8,1371517318,29,5
8,298,9,1371517318,30,6
9,298,10,1371517319,30,6


#### 4) Durée de visite d'un POI
Définir la table Visite4(seqID, poiPosition, poiID, duree) comme étant la différence entre la plus grande et la plus petite date des photos consécutives associées à un même POI.

In [ ]:
%%sql

create or replace temp view Visite4 as
  select *, (date_fin - date_debut) as duree
  from (
    select distinct seqID
      , poiPosition
      , poiID
      , min(date) over (partition by seqID, poiPosition) as date_debut
      , max(date) over (partition by seqID, poiPosition) as date_fin
    from Visite3
  );

select * from Visite4
where seqID in (298, 510)
order by seqID, poiPosition

,seqID,poiPosition,poiID,date_debut,date_fin,duree
0,298,0,22,1371514467,1371514467,0
1,298,1,7,1371514471,1371514472,1
2,298,2,23,1371516913,1371516913,0
3,298,3,28,1371516914,1371516915,1
4,298,4,1,1371517315,1371517315,0
5,298,5,29,1371517318,1371517318,0
6,298,6,30,1371517318,1371517320,2
7,298,7,8,1371517748,1371517748,0
8,298,8,29,1371517749,1371517749,0
9,298,9,6,1371518365,1371518365,0


verification: durée moyenne de visite d'un POI, pour les durée >0

In [ ]:
%%sql
select round(mean(duree)/60, 1) as duree_moyenne_en_minutes
from Visite4
where duree>0

,duree_moyenne_en_minutes
0,65.8


#### 4a) nombre moyen de visites et nombre moyen de POI dans une séquence

In [ ]:
%%sql
create or replace temp view SeqNbVisite as
select seqID, count(poiPosition) as nbVisites, count(distinct poiID) as nbPOI
from Visite4
group by seqID;

select mean(nbVisites), mean(nbPOI)
from SeqNbVisite

,mean(nbVisites),mean(nbPOI)
0,1.303285,1.255902


vérification fréquence des POI

In [ ]:
%%sql
select poiID, count(*) as poiFreqVisite
from Visite4
group by poiID
order by poiFreqVisite desc

,poiID,poiFreqVisite
0,21,848
1,22,718
2,30,598
3,16,563
4,23,557
5,7,519
6,11,505
7,28,417
8,25,406
9,4,355


#### 4b) Nombre de séquences selon leur nombre de visites. 

Afficher le nombre de séquences pour chaque nombre de visites existant. Afficher aussi, pour chaque nombre de visite, le nombre *cumulé* de séquences ayant **au moins** ce nombre de visite.


In [ ]:
%%sql
create or replace temp view T_seq_vis as
select nbVisites, count(seqID) as nbSequences
from SeqNbVisite
group by nbVisites;

select nbVisites, nbSequences, 
sum(nbSequences) over (order by nbVisites desc rows between unbounded preceding and current row) as nbre_sequences_avec_au_moins_nbVisite
from T_seq_vis
order by nbVisites

,nbVisites,nbSequences,nbre_sequences_avec_au_moins_nbVisite
0,1,5080,6057
1,2,561,977
2,3,235,416
3,4,86,181
4,5,43,95
5,6,20,52
6,7,9,32
7,8,7,23
8,9,6,16
9,10,2,10


Nombre de sequences selon leur nombre de POI distincts. 

Afficher aussi, pour chaque nombre de visite, le nombre cumulé de séquences ayant **au moins** ce nombre de POI distincts.

In [ ]:
%%sql
create or replace temp view T_seq_poi as
select nbPOI, count(seqID) as nbSequences
from SeqNbVisite
group by nbPOI;

select nbPOI, nbSequences, 
sum(nbSequences) over (order by nbPOI desc rows between unbounded preceding and current row) as nbre_sequences_avec_au_moins_nbPOI
from T_seq_poi
order by nbPOI

,nbPOI,nbSequences,nbre_sequences_avec_au_moins_nbPOI
0,1,5080,6057
1,2,642,977
2,3,216,335
3,4,60,119
4,5,33,59
5,6,9,26
6,7,9,17
7,8,4,8
8,9,2,4
9,10,1,2


Nombre de séquences ayant au moins un POI visité 2 fois

In [ ]:
%%sql
select count(*) as nbSequences
from SeqNbVisite
where nbVisites > nbPOI

,nbSequences
0,177


#### 5) Nombre de visites sur une semaine glissante
a) Définir la table Visite5a(userID, annee, mois,jour, nbVisite) : nbVisite est le nombre de visites qu'un utilisateur a fait chaque jour. 



In [ ]:
%%sql
create or replace temp view Visite5a as
  select userID
    , from_unixtime(date, 'yyyy') as annee
    , from_unixtime(date, 'MM')   as mois
    , from_unixtime(date, 'dd')   as jour
    , count(*) as nbVisite
  from user_visits
  group by userID, annee, mois, jour;

select * from Visite5a
order by userID, annee, mois, jour, nbVisite

,userID,annee,mois,jour,nbVisite
0,10007579@N00,2012,09,05,1
1,10012675@N05,2006,03,19,4
2,10012675@N05,2006,03,21,1
3,10012675@N05,2011,10,22,1
4,10012675@N05,2011,10,23,8
...,...,...,...,...,...
95,10627620@N06,2010,06,22,1
96,10627620@N06,2010,11,05,1
97,10627620@N06,2011,11,08,151
98,10627620@N06,2011,12,03,27


b) En déduire la table Visite5b(userID, anne, mois, jour, nbVisite7jours) : nbVisite7jours étant le nombre de visites effectuées sur une semaine glissante.

1.   A partir de la requête précédente :




In [ ]:
%%sql
create or replace temp view Visite5b as
  select userID
    , annee
    , mois
    , jour
    , sum(nbVisite) over (partition by userID order by timestamp(concat(annee, '-', mois, '-',jour)) range between INTERVAL '7' DAY preceding and current row) as nbVisite7Jours
  from Visite5a;

select * from Visite5b

,userID,annee,mois,jour,nbVisite7Jours
0,10007579@N00,2012,09,05,1
1,10012675@N05,2006,03,19,4
2,10012675@N05,2006,03,21,5
3,10012675@N05,2011,10,22,1
4,10012675@N05,2011,10,23,9
...,...,...,...,...,...
95,10627620@N06,2010,06,22,1
96,10627620@N06,2010,11,05,1
97,10627620@N06,2011,11,08,151
98,10627620@N06,2011,12,03,27



2.  A partir de la table *user_visits*

  2.1 **Sans** *INTERVAL '7' DAY*




In [ ]:
%%sql
select  distinct userID
  , year(timestamp(date))  as annee
  , month(timestamp(date)) as mois
  , day(timestamp(date))   as jour
  , count(timestamp(date)) over (partition by userID order by cast(from_unixtime(date) as DATE) range between 7 preceding and current row) as nbVisite7Jours
from user_visits

,userID,annee,mois,jour,nbVisite7Jours
0,10007579@N00,2012,9,5,1
1,10012675@N05,2006,3,19,4
2,10012675@N05,2006,3,21,5
3,10012675@N05,2011,10,22,1
4,10012675@N05,2011,10,23,9
...,...,...,...,...,...
95,10627620@N06,2010,6,22,1
96,10627620@N06,2010,11,5,1
97,10627620@N06,2011,11,8,151
98,10627620@N06,2011,12,3,27


  2.2 **Avec** *INTERVAL '7' DAY*

In [ ]:
%%sql
select  distinct userID
  , year(timestamp(date))  as annee
  , month(timestamp(date)) as mois
  , day(timestamp(date))   as jour
  , count(date) over (partition by userID order by timestamp(cast(from_unixtime(date) as DATE)) range between INTERVAL '7' DAY preceding and current row) as nbVisite7Jours
from user_visits


,userID,annee,mois,jour,nbVisite7Jours
0,10007579@N00,2012,9,5,1
1,10012675@N05,2006,3,19,4
2,10012675@N05,2006,3,21,5
3,10012675@N05,2011,10,22,1
4,10012675@N05,2011,10,23,9
...,...,...,...,...,...
95,10627620@N06,2010,6,22,1
96,10627620@N06,2010,11,5,1
97,10627620@N06,2011,11,8,151
98,10627620@N06,2011,12,3,27


#### 6) Déplacements entre deux POI
Définir la table Duree_Deplacement(seqID, poiPosition, poiID, deplacement). 
*deplacement* est la durée depuis la fin de la visite du POI courant jusqu'au début de visite du prochain POI dans une séquence.


In [ ]:
%%sql
select timestamp(1319327174), timestamp(1319328255), timestamp(1319328255) -timestamp(1319327174)

,1319327174,1319328255,(1319328255 - 1319327174)
0,2011-10-22 23:46:14,2011-10-23 00:04:15,0 days 00:18:01


In [ ]:
%%sql

select seqID
  , poiPosition
  , poiID
  , date
  , max(timestamp(date)) over (partition by seqID, poiPosition rows between current row and 1 following) - timestamp(date) as deplacement
  , datediff(max(timestamp(date)) over (partition by seqID, poiPosition rows between current row and 1 following), timestamp(date)) as datdiff
  from visite3


,seqID,poiPosition,poiID,date,deplacement,datdiff
0,1,0,30,1346844688,0 days 00:00:00,0
1,2,0,6,1142731848,0 days 00:06:40,0
2,2,0,6,1142732248,0 days 00:02:05,0
3,2,0,6,1142732373,0 days 00:01:12,0
4,2,0,6,1142732445,0 days 00:00:00,0
...,...,...,...,...,...,...
95,10,0,25,1215593634,0 days 00:00:16,0
96,10,0,25,1215593650,0 days 00:00:05,0
97,10,0,25,1215593655,0 days 00:00:19,0
98,10,0,25,1215593674,0 days 00:00:06,0


In [ ]:
%%sql
create or replace temp view Duree_Deplacement as
  select seqID
  , poiPosition
  , poiID
 -- , date
  , max(timestamp(date)) over (partition by seqID, poiPosition rows between current row and 1 following) - timestamp(date) as deplacement
 -- , datediff(max(timestamp(date)) over (partition by seqID, poiPosition rows between current row and 1 following), timestamp(date)) as datdiff
  from visite3;

select * from Duree_Deplacement
where seqID in (298, 510)
order by seqID, poiPosition, poiID;

,seqID,poiPosition,poiID,deplacement
0,298,0,22,0 days 00:00:00
1,298,1,7,0 days 00:00:01
2,298,1,7,0 days 00:00:00
3,298,2,23,0 days 00:00:00
4,298,3,28,0 days 00:00:01
5,298,3,28,0 days 00:00:00
6,298,4,1,0 days 00:00:00
7,298,5,29,0 days 00:00:00
8,298,6,30,0 days 00:00:01
9,298,6,30,0 days 00:00:00


## Exercice 2 : YFCC

#### Données YFCC France

In [ ]:
import zipfile


# YFCC France
web_dir = PUBLIC_DATASET + "/YFCC_dataset_extrait"
download_file(web_dir, local_dir, "yfccFrance.zip")



#unzip
if(os.path.isdir(local_dir + "/" + "yfccFrance")):
    print("file already unziped")
else:
  with zipfile.ZipFile(local_dir + "/yfccFrance.zip", 'r') as zip_ref:
    zip_ref.extractall(local_dir)


os.listdir(local_dir)


downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/YFCC_dataset_extrait/yfccFrance.zip save in : /local/data/yfccFrance.zip


['userVisits-Toro.csv', 'yfccFrance', 'POI-Toro.csv', 'yfccFrance.zip']

In [ ]:
yfcc_france = spark.read.format("parquet").load(local_dir + "/yfccFrance")
print(yfcc_france.count())
yfcc_france.printSchema()
display(yfcc_france)

2052004
root
 |-- Line: long (nullable = true)
 |-- PhotoID: long (nullable = true)
 |-- PhotoHash: string (nullable = true)
 |-- UserNSID: string (nullable = true)
 |-- UserNickname: string (nullable = true)
 |-- DateTaken: string (nullable = true)
 |-- DateUploaded: long (nullable = true)
 |-- CaptureDevice: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- UserTags: string (nullable = true)
 |-- MachineTags: string (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Accuracy: integer (nullable = true)
 |-- URL: string (nullable = true)
 |-- DownloadURL: string (nullable = true)
 |-- LicenseName: string (nullable = true)
 |-- LicenseURL: string (nullable = true)
 |-- ServerID: integer (nullable = true)
 |-- FarmID: integer (nullable = true)
 |-- Secret: string (nullable = true)
 |-- SecretOriginal: string (nullable = true)
 |-- Extension: string (nullable = true)
 |-- Marker:

,Line,PhotoID,PhotoHash,UserNSID,UserNickname,DateTaken,DateUploaded,CaptureDevice,Title,Description,...,URL,DownloadURL,LicenseName,LicenseURL,ServerID,FarmID,Secret,SecretOriginal,Extension,Marker
0,80096796,834796,dc9b7584ecb34a448540bee3b38fe85c,77922700@N00,iko,2004-09-15 19:41:25.0,1097582515,PENTAX+Corporation+PENTAX+Optio+S4,marseille,None,...,http://www.flickr.com/photos/77922700@N00/834796/,http://farm1.staticflickr.com/1/834796_9e5d1ed...,Attribution-NonCommercial-NoDerivs License,http://creativecommons.org/licenses/by-nc-nd/2.0/,1,1,9e5d1edb3a,9e5d1edb3a,jpg,0
1,64666153,5477598,4c38199a877fba534860a365bf9c97,76384935@N00,Chip_2904,2005-02-26 20:36:16.0,1109450176,None,Quiz+Night+2,None,...,http://www.flickr.com/photos/76384935@N00/5477...,http://farm1.staticflickr.com/6/5477598_d4ec28...,Attribution-NonCommercial-ShareAlike License,http://creativecommons.org/licenses/by-nc-sa/2.0/,6,1,d4ec281653,d4ec281653,jpg,0
2,82397565,5975164,2a76868b7eb18ea240d48e2941648582,70408381@N00,scot2342,2004-07-31 10:12:52.0,1110087781,NIKON+E4200,Giverny,Giverny+flowers+Monet+France,...,http://www.flickr.com/photos/70408381@N00/5975...,http://farm1.staticflickr.com/3/5975164_5ebb92...,Attribution-NonCommercial-NoDerivs License,http://creativecommons.org/licenses/by-nc-nd/2.0/,3,1,5ebb925aa1,5ebb925aa1,jpg,0
3,39862899,8060056,987ba5ffd786949496eda3a5fcf89b2,32323502@N00,Julie70,2005-03-31 12:22:18.0,1112338974,SONY+DSC-P150,They+were+all+in+it,There+will+be+always+loving+couples+in+Paris%2...,...,http://www.flickr.com/photos/32323502@N00/8060...,http://farm1.staticflickr.com/4/8060056_df5ed9...,Attribution-NonCommercial-ShareAlike License,http://creativecommons.org/licenses/by-nc-sa/2.0/,4,1,df5ed9e19b,df5ed9e19b,jpg,0
4,64976931,8916795,b1e9a0bbfd8a221830461d39a6c7e1d3,51035823282@N01,alexdecarvalho,2005-04-08 17:56:02.0,1113079413,Canon+DIGITAL+IXUS+40,Grupo+Corpo,None,...,http://www.flickr.com/photos/51035823282@N01/8...,http://farm1.staticflickr.com/8/8916795_11c551...,Attribution-NonCommercial-ShareAlike License,http://creativecommons.org/licenses/by-nc-sa/2.0/,8,1,11c551a24b,11c551a24b,jpg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,67598393,190189307,e04ae5f6aaec837234c28ff81c20d27b,38996820@N00,clarque,2006-06-27 02:43:02.0,1152989336,OLYMPUS+CORPORATION+C765UZ,Hike+near+Pralognan,None,...,http://www.flickr.com/photos/38996820@N00/1901...,http://farm1.staticflickr.com/76/190189307_6fd...,Attribution-NoDerivs License,http://creativecommons.org/licenses/by-nd/2.0/,76,1,6fd7baff97,6fd7baff97,jpg,0
96,94436396,190670498,f360dc5abc47711ce3d54953f52483,95285464@N00,leguan001,2003-04-30 08:18:40.0,1153049290,Canon+PowerShot+S40,Hohk%C3%B6nigsburg+%28Ch%C3%A2teau+du+Haut-K%C...,None,...,http://www.flickr.com/photos/95285464@N00/1906...,http://farm1.staticflickr.com/71/190670498_6f5...,Attribution-NonCommercial-NoDerivs License,http://creativecommons.org/licenses/by-nc-nd/2.0/,71,1,6f51679d0a,6f51679d0a,jpg,0
97,93144642,191107677,a0ae43beae4f87af92905d402e77f768,65648243@N00,Laurent+Paris11,2006-07-14 19:26:20.0,1153086075,NIKON+CORPORATION+NIKON+D200,_DSC0011+-+Version+2,None,...,http://www.flickr.com/photos/65648243@N00/1911...,http://farm1.staticflickr.com/65/191107677_229...,Attribution-NonCommercial-NoDerivs License,http://creativecommons.org/licenses/by-nc-nd/2.0/,65,1,229652f913,229652f913,jpg,0
98,82701618,193290427,7d16a5ab81a9cc605391cd1a7eeb8ceb,40286210@N00,Bryce+Edwards,2006-07-13 20:25:28.0,1153305586,SONY+DSC-F828,DSC09269,View+from+Terrace%2C+La+Cite+Radieuse%2C+Le+Co...,...,http://www.flickr.com/photos/40286210@N00/1932...,http://farm1.staticflickr.com/72/193290427_f63...,Attribution License,http://creativecommons.org/licenses/by/2.0/,72,1,f6332a248b,f6332a248b,jpg,0


#### Question 1

Extraire de YFCC les sequences de points des utilisateurs telles que les conditions suivantes soient vérifiées :

Une séquence ne peut pas couvrir plusieurs jours. Si un utilisateur a pris des photos pendant plusieurs jours consécutifs, cela forme plusieurs séquences.

Une séquence doit contenir au moins 3 points distincts.

Chaque point doit être associé à au moins 3 utilisateurs.

On peut supposer que deux photos prises à deux positions GPS très proches (moins de *d* mètres entre les deux positions) correspondent à un même point. 


In [ ]:
yfcc_france.createOrReplaceTempView("yfcc_france")

In [ ]:
%%sql
create or replace temp view Yfcc as
select UserNSID, cast(DateTaken as date) as DateTaken, Longitude, Latitude
from yfcc_france;

select *
from Yfcc

,UserNSID,DateTaken,Longitude,Latitude
0,77922700@N00,2004-09-15,5.371059,43.284172
1,76384935@N00,2005-02-26,6.832551,45.573586
2,70408381@N00,2004-07-31,1.524825,49.077106
3,32323502@N00,2005-03-31,2.346675,48.852306
4,51035823282@N01,2005-04-08,2.304210,48.866394
...,...,...,...,...
95,38996820@N00,2006-06-27,6.701316,45.349491
96,95285464@N00,2003-04-30,7.343984,48.249477
97,65648243@N00,2006-07-14,0.706644,49.172443
98,40286210@N00,2006-07-13,5.368194,43.321678


La table **T_point_users** contient les points associés à au moins 3 utilisateurs, ainsi qu'un identifiant pour chacun des points.

In [ ]:
%%sql
create or replace temp view T_point_users as
select point_i, Longitude, Latitude
from (
  select Longitude, Latitude, count(UserNSID) as count_users, row_number() over (order by Longitude, Latitude) as point_i
  from Yfcc
  group by Longitude, Latitude
)
where count_users > 2;

select *
from T_point_users

,point_i,Longitude,Latitude
0,16,-5.140571,48.449726
1,19,-5.139970,48.448959
2,24,-5.139284,48.449471
3,34,-5.132675,48.449699
4,43,-5.130752,48.459846
...,...,...,...
95,805,-4.772100,48.330261
96,807,-4.772036,48.330307
97,819,-4.771853,48.330521
98,821,-4.771842,48.330692


La table **T1** contient les attributs pertinents et où chaque point est associé à au moins 3 utilisateurs.

In [ ]:
%%sql
create or replace temp view T1 as
select distinct y.UserNSID, y.DateTaken, y.Longitude, y.Latitude, pu.point_i
from Yfcc y, T_point_users pu
where y.Longitude == pu.Longitude and y.Latitude == pu.Latitude;

select *
from T1

,UserNSID,DateTaken,Longitude,Latitude,point_i
0,97229645@N06,2010-07-14,-5.140571,48.449726,16
1,9654398@N07,2008-08-12,-5.139970,48.448959,19
2,21133169@N07,2003-08-09,-5.139284,48.449471,24
3,23447827@N02,2007-07-15,-5.132675,48.449699,34
4,23447827@N02,2007-05-26,-5.132675,48.449699,34
...,...,...,...,...,...
95,23203406@N06,2008-06-01,-4.778881,48.444519,605
96,23203406@N06,2007-03-11,-4.778881,48.444519,605
97,23203406@N06,2008-06-29,-4.778881,48.444519,605
98,42523628@N05,2009-09-13,-4.778795,48.353397,608


La table **T2** contient des séquences d'au moins 3 points distincts en fonction de l'utilisateur et de la date, et où un point est associé à au moins 3 utilisateurs.

In [ ]:
%%sql

create or replace temp view T2 as
select *
from (
  select UserNSID, DateTaken, collect_list(distinct point_i) as Sequence
  from T1
  group by UserNSID, DateTaken
  order by UserNSID, DateTaken
)
where cardinality(Sequence) > 2;

select *
from T2

,UserNSID,DateTaken,Sequence
0,10012675@N05,2007-08-24,"[193595, 197610, 204096, 261759, 262320, 293229]"
1,100142027@N04,2013-02-09,"[240848, 294478, 305883]"
2,100297489@N07,2013-04-06,"[144258, 144317, 145985, 146361]"
3,10058483@N00,2006-07-14,"[208284, 265495, 313138]"
4,10065047@N00,2013-11-05,"[417747, 453851, 455142, 459324]"
...,...,...,...
95,10297518@N00,2009-01-31,"[107858, 115369, 117603]"
96,10297518@N00,2010-06-10,"[185988, 186005, 224522, 257462]"
97,10297518@N00,2011-03-12,"[209892, 244275, 313140]"
98,10297518@N00,2011-03-23,"[239555, 240019, 254004]"


#### Question 2

Proposer un exemple d'analyse des séquences obtenues à la question précédente suivante 3 dimensions dont au moins une doit avoir au moins 3 niveaux.